In [2]:
import sys
sys.path.insert(0,'../../')

from src.config import app_config

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

ENGINE  = create_engine(app_config.SQLALCHEMY_DATABASE_URI, echo=False, pool_size=20, max_overflow=0)

##  Do autotrade

### Fetch autotrade symbols

In [4]:
from src.service import DataService

rows = []
with Session(ENGINE) as session:
    # Fetch the currency pairs that are enabled for autotrade
    rows = DataService().get_active_bots(session)

    print(rows)

[
            CurrencyPairConfig(
                id=1,
                user_id=1,
                currency_pair=BTCUSDT,
                exchange=ExchangeType.BINANCE,
                interval=Interval.MINUTE_1,
                strategy=StrategyType.BOLLINGER_BANDS,
                limit=1000.00000000,
                stop_loss=0.3,
                take_profit=0.3,
                earnings=0.0,
                allocated_balance=1000.0,
                currency_free=1000.0,
                currency_locked=0.0,
                asset_free=0.0,
                asset_locked=0.0,
                is_active=True,
                is_decommissioned=False,
                is_simulated=True,
                updated_at=2023-05-08 11:48:32

                signal=
            SignalModel(
                id=1,
                signal=Signal.HOLD,
                last_trade_time=2023-03-16 00:33:05,
                updated_at=2023-03-16 00:33:05
            )
        ,
                orders=[],
    

**Generate signal**

In [3]:
from src.service import SignalService, DataService, ExchangeService

row = rows[0]
id = row.get('id')
currency_pair_config_id = row.get('currency_pair_config_id')
print('currency_pair_config_id', currency_pair_config_id)

with Session(ENGINE) as session:
    # Fetch the user's autotrade config for the currency pair
    currency_pair_config = DataService().get_currency_pair_config(session, currency_pair_config_id)
    user_id = currency_pair_config.get('user_id')
    exchange_type = currency_pair_config.get('exchange')
    print('user_id', user_id)
    print('exchange_type', exchange_type)

    # Fetch api keys
    api_key_pair = DataService().get_api_key(session, user_id, exchange_type)
    print('api_key_pair', api_key_pair)

    # Fetch the exchange service
    exchange = ExchangeService().get_exchange(exchange_type, **api_key_pair)
    print('exchange', exchange)

    # Generate the signal
    signal = SignalService(exchange, currency_pair_config).generate_signal()
    print('signal', signal)

    # Update the autotrade table with the signal
    DataService().update_autotrade_signal(session, row.get('id'), signal.name)

currency_pair_config_id 1
user_id 1
exchange_type binance
api_key_pair {'api_secret': 't9jIRnruWjoBAyzy7tlCrxMQ23McF6IbcduSl6IaaprzH8cLtjqDgnBDmFzLhFPd', 'api_key': 'xLNK8OVHg552tLqZsRRPQfD1l6LEkpj02xtrB2tlamLysSyRRJV07kqu9I6sav4V'}
exchange <exchange.binance.Binance object at 0x11decbc10>
signal Signal.SELL


**Do Buy/Sell action**

In [3]:
from service import TraderService, DataService, ExchangeService
from enums import Interval, Signal
from datetime import datetime

row = rows[0]
id = row.get('id')
last_trade_time = row.get('last_trade_time')
last_trade_time = datetime.strptime(last_trade_time, '%Y-%m-%dT%H:%M:%S')
currency_pair_config_id = row.get('currency_pair_config_id')
print('last_trade_time', last_trade_time)
print('currency_pair_config_id', currency_pair_config_id)

with Session(ENGINE) as session:
    # Fetch the user's autotrade config for the currency pair
    currency_pair_config = DataService().get_currency_pair_config(session, currency_pair_config_id)
    interval = Interval(currency_pair_config.get('interval'))
    print('interval', interval)

    # Check if the last trade was made more than the interval ago
    interval_seconds = DataService().get_interval_seconds(interval)
    print('interval_seconds', interval_seconds)

    now = datetime.now()
    print('now', now)
    
    if (now - last_trade_time).total_seconds() < interval_seconds:
        print('Not enough time has passed since last trade')
    
    user_id = currency_pair_config.get('user_id')
    exchange_type = currency_pair_config.get('exchange')
    
    # Fetch api keys
    api_key_pair = DataService().get_api_key(session, user_id, exchange_type)
    print('api_key_pair', api_key_pair)

    # Fetch the exchange service
    exchange = ExchangeService().get_exchange(exchange_type, **api_key_pair)
    print('exchange', exchange)

    signal = Signal[row.get('signal')]
    print(signal)

    # Trade
    trader = TraderService(exchange, currency_pair_config)

    # order = trader.create_order(session, Signal.BUY)
    order = trader.create_order(session, Signal.SELL)
    print('order', order)

    if currency_pair_config.get('is_simulated') == 1 and order is not None:
        trader.update_order(session, order)


last_trade_time 2023-03-16 00:33:05
currency_pair_config_id 1
interval Interval.MINUTE_1
interval_seconds 60
now 2023-03-16 00:53:15.389660
api_key_pair {'api_key': 'xLNK8OVHg552tLqZsRRPQfD1l6LEkpj02xtrB2tlamLysSyRRJV07kqu9I6sav4V', 'api_secret': 't9jIRnruWjoBAyzy7tlCrxMQ23McF6IbcduSl6IaaprzH8cLtjqDgnBDmFzLhFPd'}
exchange <exchange.binance.Binance object at 0x120d43460>
Signal.HOLD
order {'exchange': 'binance', 'status': 'new', 'updated_at': '2023-03-16T00:53:16', 'order_id': 'b56aa3511ab44db696c83c985dcc5c03', 'id': 18, 'currency_pair': 'BTCUSDT', 'side': 'SELL', 'amount': 24293.63, 'autotraded': True, 'type': 'limit', 'cost': 1.0, 'limit_price': 24293.63, 'created_at': '2023-03-16T00:53:16', 'last_price': 24293.63, 'user_id': '1'}
sell update


In [2]:
from service import DataService

with Session(ENGINE) as session:
    # fetch open orders
    open_orders = DataService().get_open_orders(session, '1')
    print(open_orders)

[{'id': 5, 'user_id': '1', 'currency_pair': 'ETHUSDT', 'order_id': '5', 'exchange': 'binance', 'cost': 1000.0, 'last_price': 10.0, 'amount': 10.0, 'side': 'BUY', 'status': 'new', 'type': 'market', 'limit_price': None, 'autotraded': False, 'created_at': datetime.datetime(2023, 3, 16, 0, 33, 5), 'updated_at': datetime.datetime(2023, 3, 16, 0, 33, 5)}, {'id': 6, 'user_id': '1', 'currency_pair': 'BTCUSDT', 'order_id': '6', 'exchange': 'binance', 'cost': 1000.0, 'last_price': 10.0, 'amount': 10.0, 'side': 'SELL', 'status': 'new', 'type': 'market', 'limit_price': None, 'autotraded': False, 'created_at': datetime.datetime(2023, 3, 16, 0, 33, 5), 'updated_at': datetime.datetime(2023, 3, 16, 0, 33, 5)}, {'id': 17, 'user_id': '1', 'currency_pair': 'BTCUSDT', 'order_id': '4bfa4d961b2d443ba25f56da878f7e85', 'exchange': 'binance', 'cost': 1.0, 'last_price': 24277.12, 'amount': 24277.12, 'side': 'SELL', 'status': 'new', 'type': 'limit', 'limit_price': 24277.12, 'autotraded': True, 'created_at': date